 Organizar arquivos por Ano.
 Copia os arquivos e adiciona na pasta do respectivo ano ao que o arquivo foi criado. Organiza os arquivos e copia a estrutura de diretórios que contém os arquivos.

In [11]:
#Importa as bibliotecas necessárias
import datetime
import os
import shutil
import time

In [12]:
#Define o endereço de origem dos arquivos.
#path_origem = 'C:\\Users\\charlessodre\\Documents\\PastaVM\\Estudos\\OrganizadorArquivos\\source_img'
path_origem = 'D:\\FOTOS\\'
#path_origem = 'D:\\FOTOS\\'

#Define o endereço de destino dos arquivos.
#path_destino = 'C:\\Users\\charlessodre\\Documents\\PastaVM\\Estudos\\OrganizadorArquivos\\DESTINO'
#path_destino = 'D:\\FOTOS_POR_ANO\\'
path_destino = 'D:\\TESTE\\'

In [13]:
# Define a Função para Cria um diretório, se não existir
def criar_diretorio(path, recursivo=False):
    if not os.path.exists(path):
        #print(novo_dir)
        if recursivo:
            os.makedirs(path)
        else:
            os.mkdir(path)

In [14]:
# Define a Função que retorna o ano do arquivo.
def ano_arquivo(file):
    return datetime.datetime.fromtimestamp(os.stat(file).st_mtime).year

In [15]:
#Define a função que retorna os segundos no formato hh:mm:ss
def formata_segundos_hhmmss(segundos):
    return time.strftime('%H:%M:%S', time.gmtime(segundos))

In [16]:
#Define a função que retorna da hora atual do sistema.
def get_hora_atual():
    return time.strftime("%H:%M:%S")

In [17]:
#Define a função que grava um arquivo TXT a partir de uma lista.
def grava_arquivo_log(lista_log, path_destino_log, modo_gravacao='w'):
    
    log_copia_arquivos = os.path.join(path_destino_log, "log_copia_arquivos.txt")
    
    with open(log_copia_arquivos, modo_gravacao) as f:
        for s in lista_log:
            f.write(str(s) + '\n')

In [18]:
# Denfine a função que retorna um dicionário com a quantidade arquivos por diretório e a frequncia dos anos. 
def dic_arquivos_anos(path):
    
    qtd_arq_dir = {}

    for root, dirs, files in os.walk(path):
                              
        freq_ano = {}

        split_dir = root.split('\\')
        nome_dir = split_dir[len(split_dir)-1]

        anos_arq = [ano_arquivo(os.path.join(root, f)) for f in files]

        for ano in anos_arq:
            freq_ano[ano] = anos_arq.count(ano)


        qtd_arq_dir[root] = len(files), freq_ano, nome_dir
        
        #print("Root:", root)
        #print('Diretório:', nome_dir)
        #print('Qtd Arq: ', set(anos_arq))
        #print('Freq Ano: ',freq_ano)
        #print('----------------------------')
       

    return qtd_arq_dir

In [24]:
#Define a Função principal. 
def main ():
    
    
    #Variáveis de controle para debug
    habilitar_prints = False
    habilitar_gravacao_log = True
    habilitar_criacao_diretorios = False
    habilitar_copia_arquivos = False
    
    #Lista com as mensagens
    log_execucao = []
    
    msg_inicio_execucao = '----------------------------------------------- Início Execução ----------------------------------------------- '
        
    log_execucao.append(msg_inicio_execucao)
    log_execucao.append('\n')
       
    #Data e hora atual
    hora_inicio = get_hora_atual()
       
    #Define o tempo de inicio da execução
    inicio = time.time()
    
    #Anos dos arquivos
    arquivos_anos = dic_arquivos_anos(path_origem)
    total_arquivos = 0
    
    #Percorre todos os diretórios do dicionário
    for key in arquivos_anos:      
        #Obtêm o diconário da frequencia de anos.
        dic_anos = arquivos_anos[key][1]
        
        #Verifica se o diretório está vazio, se estiver passa para o proximo.
        if len(dic_anos) == 0:
            continue
        
        #Obtêm a chave com maior valor
        anoComMaisArquivos = max(dic_anos, key=dic_anos.get)
        
        msg_inicio_copia = 'Copiando arquivos do diretório:[ %s ]'% key
        msg_ano_ocorrencia = 'Ano com maior acorrência: [ %d ]' % anoComMaisArquivos
        
        log_execucao.append(msg_inicio_copia)
        log_execucao.append(msg_ano_ocorrencia)
        
        if habilitar_prints:
            print(msg_inicio_copia)
            print(msg_ano_ocorrencia)
             
        dir_origem = os.path.abspath(key)
        msg_origem = 'Origem  <------ %s' % dir_origem
        
        log_execucao.append(msg_origem)
            
        if habilitar_prints:
            print(msg_origem)  
        
        # Monta o PATH de destino dos arquivos.
        dir_destino_ano = os.path.join(path_destino, str(anoComMaisArquivos)+'\\')      
        dir_destino = os.path.abspath(os.path.join(dir_destino_ano, key.replace(path_origem, dir_destino_ano) ))     
             
        msg_destino =  'Destino ------> %s' % dir_destino   
        
        log_execucao.append(msg_destino)
               
        if habilitar_prints:
            print(msg_destino)
        
        #Cria os diretórios
        if habilitar_criacao_diretorios:
            criar_diretorio(dir_destino, recursivo=True)
             
        arquivos = os.listdir(dir_origem)
               
        qtd_arq = 0
        
        #Percorre todos os arquivos do diretório
        for f in arquivos:
            
            path_arquivo = os.path.join(dir_origem, f)
            
            if(os.path.isfile(path_arquivo)):
                qtd_arq +=1
                total_arquivos += 1
                msg_log_arquivo = "Arq %d de %d |Nome: %s |Origem: %s | Hora Log: %s"   % (qtd_arq, arquivos_anos[key][0], f, path_arquivo, get_hora_atual())
                
                log_execucao.append(msg_log_arquivo)
               
                if habilitar_prints:
                    print(msg_log_arquivo) 
               
                #Copia os arquivos
                if habilitar_copia_arquivos:
                    shutil.copy2(path_arquivo, dir_destino)
                    
        
        log_execucao.append('\n')
        
        if habilitar_prints:   
            print('\n')
    
    #Define o tempo final da execução
    fim = time.time()
    
    #Mensagens de log 
    msg_fim_execucao = '----------------------------------------------- Fim Execução ----------------------------------------------- '
    
    msg_total_aquivos_origem = 'Total de diretórios copiados: %d' % len(arquivos_anos)
    msg_total_aquivos_copiados = 'Total de Arquivos copiados: %d' % total_arquivos
    msg_hora_inicio = 'Hora iníco: %s' % hora_inicio
    msg_hora_fim = 'Hora fim: %s'% get_hora_atual()
    msg_tempo_execucao = 'Tempo da Execução: %s' % formata_segundos_hhmmss(fim - inicio)
    
    log_execucao.append(msg_fim_execucao) 
    log_execucao.append('\n')
    log_execucao.append(msg_total_aquivos_origem)
    log_execucao.append(msg_total_aquivos_copiados)
    log_execucao.append(msg_hora_inicio)
    log_execucao.append(msg_hora_fim)  
    log_execucao.append(msg_tempo_execucao)
    
    if habilitar_prints:
        print(msg_fim_execucao)
        print(msg_total_aquivos_origem)
        print(msg_total_aquivos_copiados)
        print(msg_hora_inicio)
        print(msg_hora_fim)
        print(msg_tempo_execucao)
   
    #Grava o arquivo de Log.
    if habilitar_gravacao_log:
        grava_arquivo_log(log_execucao, path_destino)

In [25]:
#Executa função principal
main()